In [1]:
import os
import re
import sys
import uuid
import hashlib
import socket
import requests
import json
import time
import cProfile
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_v1_5
from time import time
from urllib.parse import urlparse
from flask import request
from operator import itemgetter
from datetime import datetime, timezone
from pyp2p.net import *

from config import Config

In [2]:
host = 'http://0.0.0.0:5000'

# # Coinbase
# wallet = requests.post(host+'/wallet/new', json={'password': '12345'}).content
# wallet

In [12]:
host = 'http://0.0.0.0:5001'
accounts = requests.get(host+'/wallet/getAccounts').content
accounts

b'{"ACCOUNTS:":"80035d710063626c6f636b636861696e0a4163636f756e740a7101298171027d7103285807000000616464726573737104583e0000007a303178303031383730303263353462316431653465333936376132613561653364636361633239663133303063386336363832626333623932326535637105580700000062616c616e636571064700000000000000005809000000626c6f636b4861736871075840000000643833376631303663636637323935663732323033396339636138356237336337623361393635326536313438613466626131353735356237633438663964637108580900000076616c696448617368710958380000003062393231333665363134623365613231633435383164663138366432343132353630363862323363316433653735363236383631626338710a5803000000736c74710b58200000003034333737316561363239323463663438323062613665333161393562626335710c7562612e"}\n'

In [13]:
import pickle
accounts = json.loads(accounts)['ACCOUNTS:']
accounts = pickle.loads(bytes.fromhex(accounts))
accounts

In [14]:
account = accounts[0]
account.address

'z01x00187002c54b1d1e4e3967a2a5ae3dccac29f1300c8c6682bc3b922e5c'

In [3]:
wallet = requests.post(host+'/wallet/new', json={'password': 'qwerty'}).content

In [6]:
wallet

b'{"ADDRESS":"z01x00acae0943989d5aed79a4cb334f940ec673dfe8a338986d91d821e569"}\n'

In [4]:
balances = json.loads(requests.post(host+'/wallet/getBalance', json={'address': json.loads(login)['ADDRESS']}).content)['BALACNES']
# remainBalance = [balance['balance'] for balance in balances if balance['token']=='USDT'][0]
# remainBalance
balances

NameError: name 'login' is not defined

In [4]:
pools = requests.get(host+'/pools/getPools').content
pools


b'{"MSG": [{"name": "Tether", "symbol": "USDT", "poolAddress": "z01x006538614d4e85b942bf093d9b4e3543713f447771e5c6d0d4c2980b42", "poolBalance": 5731.300000000002, "accounts": {"z01x008de7af2b8b9f4bfc0b989897b4ec564ab776812c00861f613d6333a6": 998181.5900000003, "z01x00ea64f3ad8569afc06bce1b72a2aa8201204f70f2c3c2f416edf5ec3e": 996087.1100000001}, "blockHash": "94c3439baa95fc878320ac8648e933f23d68640de0ce27304a94e1d9c1562deb", "validHash": "49d9c16ff61273594e01bc6035f74fb99510b7bb0fe79d7771fb5513"}]}'

In [2]:
host = 'http://0.0.0.0:5000'

name = 'Tether' #Tether Ether
symbol = 'usdt' #usdt eth
pool = requests.post(host+'/pools/createPool', json={'name': name, 'symbol': symbol}).content

login = requests.post(host+'/wallet/login', json={'password': '12345'}).content
# mineReq = requests.get(host+'/mine').content
# mineReq = requests.get(host+'/mine').content
# mineReq = requests.get(host+'/mine').content
# mineReq = requests.get(host+'/mine').content
# mineReq = requests.get(host+'/mine').content
# mineReq = requests.get(host+'/mine').content
# mineReq = requests.get(host+'/mine').content
# mineReq = requests.get(host+'/mine').content
# mineReq = requests.get(host+'/mine').content
# mineReq = requests.get(host+'/mine').content

In [78]:
mineReq = requests.get(host+'/mine').content
mineReq

b'{"index":12,"message":"New Block Forged","pool_syncing":"Tx pool synced among 0 nodes","previous_hash":"a2e90700e04b659d4cd87b61402873fcf88ec655294123955dc861d3c2295bd8","proof":62,"transactions":[{"comissionAmount":2.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"z01x00bc5a3262dc9f9537e4950761a40b30a51055be4bed6b62df7f01f369","sendAmount":5,"sender":"z01x00017cb4893e68be741543c24527d36d8838f89dbd8152f0405f632c2d","symbol":"zsh","timestamp":1630578549.890106,"tradeTxId":null},{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"z01x00017cb4893e68be741543c24527d36d8838f89dbd8152f0405f632c2d","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1630578554.680469,"tradeTxId":null}]}\n'

In [33]:
ordersList = json.loads(requests.get(host+'/getTradeOrders').content)['tradeOrders']
ordersList

[{'comissionAmount': 2.0,
  'get': 'zsh',
  'getVol': 0.5,
  'price': 12,
  'send': 'usdt',
  'sendVol': 6.600000000000001,
  'sender': 'z01x00631e4f56fed99ce386725ada1ac2d17087481e2cc961f8a57dc34e0e',
  'symbol': 'zsh/usdt',
  'timestamp': 1630564084.443995,
  'tradeTxId': '1285345b45d974234efb7e9426b7da06f395b4937c56b917f3aed7507477a23c'},
 {'comissionAmount': 2.0,
  'get': 'zsh',
  'getVol': 6.5,
  'price': 13,
  'send': 'usdt',
  'sendVol': 84.65,
  'sender': 'z01x0082c3cabb3899b24010a542ce9018d0141c0126c642f1bca12add505a',
  'symbol': 'zsh/usdt',
  'timestamp': 1630564678.976997,
  'tradeTxId': '706abbbbed47629d065a5e992de4d7f985ccb24f4a58f182c04ad3e27974805c'}]

In [16]:
mineReq = requests.get(host+'/mine').content
mineReq

b'{"index":6,"message":"New Block Forged","pool_syncing":"Tx pool synced among 0 nodes","previous_hash":"cad857bc8304c5094d4818999c7f4a89aab407d9f7fd9438e84e9d58598ee586","proof":33,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"z01x00bcfd3c8de490ead704dc4610efb6b7894cc84ce10cb697c493462d33","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1630516708.377076,"tradeTxId":null},{"comissionAmount":0.02,"contract":"zsh","price":10,"recieveAmount":10,"recipient":"z01x00bcfd3c8de490ead704dc4610efb6b7894cc84ce10cb697c493462d33","sendAmount":1,"sender":"z01x00bcfd3c8de490ead704dc4610efb6b7894cc84ce10cb697c493462d33","symbol":"zsh/usdt","timestamp":1630516708.377131,"tradeTxId":"2cb6946ebe22e78c392b336fff01e75988888a811ce8d8990e105c3890e49939"},{"comissionAmount":0.02,"contract":"usdt","price":10,"recieveAmount":1,"recipient":"z01x00bcfd3c8de490ead704dc4610efb6b7894cc84ce10cb6

In [9]:
buyTx = {
        'type': 'trade',
        'sender': json.loads(wallet)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 9.93,
        'send': 'usdt',
        'sendVol': 9.93,
        'get': 'zsh',
        'getVol': 1.0,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=buyTx).content

In [27]:
ordersList = requests.get(host+'/getTradeOrders').content
ordersList

b'{"tradeOrders":[{"comissionAmount":2.0,"get":"usdt","getVol":6.467049118441537e-17,"price":9.67,"send":"zsh","sendVol":0.0,"sender":"z01x00c307e4731e05816fccbd36ee630babc8c04c85485396e0a4d7c183d7","symbol":"zsh/usdt","timestamp":1630403036.819795,"tradeTxId":"a8854ba574cdd145b8eb1038e32de1d6b1aef544c601e7bfd94b8e00617a84e8"},{"comissionAmount":2.0,"get":"zsh","getVol":0.09999999999999998,"price":9.68,"send":"usdt","sendVol":0.9679999999999997,"sender":"z01x00c307e4731e05816fccbd36ee630babc8c04c85485396e0a4d7c183d7","symbol":"zsh/usdt","timestamp":1630403093.317493,"tradeTxId":"7701340acf546157cf44cf782702ad4405c1cd71fba2387a26ab6dcf115a2c23"},{"comissionAmount":2.0,"get":"zsh","getVol":0.2,"price":9.87,"send":"usdt","sendVol":1.974,"sender":"z01x00c307e4731e05816fccbd36ee630babc8c04c85485396e0a4d7c183d7","symbol":"zsh/usdt","timestamp":1630403129.486398,"tradeTxId":"a03fecb39f744d6bc261ca2b076521c14989acdd407eb935abb9475a1d13c492"},{"comissionAmount":2.0,"get":"usdt","getVol":1.974,"

In [30]:
olist = json.loads(ordersList)['tradeOrders']
getVol = olist[0]['getVol']
getVol

6.467049118441537e-17

In [32]:
getVol < 0.000099

True

### Full non-native tokens transaction Test

In [25]:
host = 'http://0.0.0.0:5000'

# Coinbase
wallet = requests.post(host+'/wallet/new', json={'password': '12345'}).content
wallet = requests.post(host+'/wallet/new', json={'password': 'qwerty'}).content

name = 'Tether' #Tether Ether
symbol = 'usdt' #usdt eth
pool = requests.post(host+'/pools/createPool', json={'name': name, 'symbol': symbol}).content

login = requests.post(host+'/wallet/login', json={'password': '12345'}).content
mineReq = requests.get(host+'/mine').content
mineReq = requests.get(host+'/mine').content
mineReq = requests.get(host+'/mine').content
mineReq = requests.get(host+'/mine').content
mineReq = requests.get(host+'/mine').content
mineReq = requests.get(host+'/mine').content
mineReq = requests.get(host+'/mine').content
mineReq = requests.get(host+'/mine').content
mineReq = requests.get(host+'/mine').content
mineReq = requests.get(host+'/mine').content

с = 0
sellTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 10.93+с,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 10.93+с,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=sellTx).content
с+=1
sellTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 10.93+с,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 10.93+с,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=sellTx).content
с+=1
sellTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 10.93+с,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 10.93+с,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=sellTx).content
с+=1
sellTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 10.93+с,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 10.93+с,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=sellTx).content
с+=1
sellTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 10.93+с,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 10.93+с,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=sellTx).content
с+=1
sellTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 10.93+с,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 10.93+с,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=sellTx).content
с+=1
sellTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 10.93+с,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 10.93+с,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=sellTx).content
с+=1
sellTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 10.93+с,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 10.93+с,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=sellTx).content
с+=1
sellTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 10.93+с,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 10.93+с,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=sellTx).content
с+=1
sellTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 10.93+с,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 10.93+с,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=sellTx).content


с=0
login = requests.post(host+'/wallet/login', json={'password': 'qwerty'}).content

buyTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 25.25+с,
        'send': 'usdt',
        'sendVol': 252.5+с,
        'get': 'zsh',
        'getVol': 10.0,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=buyTx).content
с-=1
buyTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 27,
        'send': 'usdt',
        'sendVol': 270,
        'get': 'zsh',
        'getVol': 10.0,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=buyTx).content
# с-=1
# buyTx = {
#         'type': 'trade',
#         'sender': json.loads(login)['ADDRESS'],
#         'symbol': 'zsh/usdt',
#         'price': 9.93+с,
#         'send': 'usdt',
#         'sendVol': 9.93+с,
#         'get': 'zsh',
#         'getVol': 1.0,
#         'comissionAmount':2
#     }
# txReq = requests.post(host+'/transactions/new', json=buyTx).content
# с-=1
# buyTx = {
#         'type': 'trade',
#         'sender': json.loads(login)['ADDRESS'],
#         'symbol': 'zsh/usdt',
#         'price': 9.93+с,
#         'send': 'usdt',
#         'sendVol': 9.93+с,
#         'get': 'zsh',
#         'getVol': 1.0,
#         'comissionAmount':2
#     }
# txReq = requests.post(host+'/transactions/new', json=buyTx).content
# с-=1
# buyTx = {
#         'type': 'trade',
#         'sender': json.loads(login)['ADDRESS'],
#         'symbol': 'zsh/usdt',
#         'price': 9.93+с,
#         'send': 'usdt',
#         'sendVol': 9.93+с,
#         'get': 'zsh',
#         'getVol': 1.0,
#         'comissionAmount':2
#     }
# txReq = requests.post(host+'/transactions/new', json=buyTx).content
# с-=1
# buyTx = {
#         'type': 'trade',
#         'sender': json.loads(login)['ADDRESS'],
#         'symbol': 'zsh/usdt',
#         'price': 9.93+с,
#         'send': 'usdt',
#         'sendVol': 9.93+с,
#         'get': 'zsh',
#         'getVol': 1.0,
#         'comissionAmount':2
#     }
# txReq = requests.post(host+'/transactions/new', json=buyTx).content
# с-=1
# buyTx = {
#         'type': 'trade',
#         'sender': json.loads(login)['ADDRESS'],
#         'symbol': 'zsh/usdt',
#         'price': 9.93+с,
#         'send': 'usdt',
#         'sendVol': 9.93+с,
#         'get': 'zsh',
#         'getVol': 1.0,
#         'comissionAmount':2
#     }
# txReq = requests.post(host+'/transactions/new', json=buyTx).content
# с-=1
# buyTx = {
#         'type': 'trade',
#         'sender': json.loads(login)['ADDRESS'],
#         'symbol': 'zsh/usdt',
#         'price': 9.93+с,
#         'send': 'usdt',
#         'sendVol': 9.93+с,
#         'get': 'zsh',
#         'getVol': 1.0,
#         'comissionAmount':2
#     }
# txReq = requests.post(host+'/transactions/new', json=buyTx).content
# с-=1
# buyTx = {
#         'type': 'trade',
#         'sender': json.loads(login)['ADDRESS'],
#         'symbol': 'zsh/usdt',
#         'price': 9.93+с,
#         'send': 'usdt',
#         'sendVol': 9.93+с,
#         'get': 'zsh',
#         'getVol': 1.0,
#         'comissionAmount':2
#     }
# txReq = requests.post(host+'/transactions/new', json=buyTx).content
# с-=1
# buyTx = {
#         'type': 'trade',
#         'sender': json.loads(login)['ADDRESS'],
#         'symbol': 'zsh/usdt',
#         'price': 9.93+с,
#         'send': 'usdt',
#         'sendVol': 9.93+с,
#         'get': 'zsh',
#         'getVol': 1.0,
#         'comissionAmount':2
#     }
# txReq = requests.post(host+'/transactions/new', json=buyTx).content


mineReq = requests.get(host+'/mine').content
mineReq

b'{"index":12,"message":"New Block Forged","pool_syncing":"Tx pool synced among 0 nodes","previous_hash":"82dea2a9c4ac776e716a92a825190417125f5e7e14f9c078e4ae0ac70881dbb0","proof":62,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"z01x0081ca38a9e3270c25897ce76b2eed56e58d59dd9b4f82860551ecff52","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1630498714.360835,"tradeTxId":null},{"comissionAmount":0.02,"contract":"zsh","price":10.93,"recieveAmount":10.93,"recipient":"z01x005ea5ca02a87f2dcf8685c8ae053d2cafb4c1e8b7f0519fb6676e1921","sendAmount":1.0,"sender":"z01x0081ca38a9e3270c25897ce76b2eed56e58d59dd9b4f82860551ecff52","symbol":"zsh/usdt","timestamp":1630498714.36091,"tradeTxId":"bb17ecdb897ed8d7252e470f8a44632c9ca15459ac954aa32e99d1f95257f162"},{"comissionAmount":0.0008096296296296297,"contract":"usdt","price":10.93,"recieveAmount":1.0,"recipient":"z01x0081ca38a9e3270c

In [26]:
ordersList = json.loads(requests.get(host+'/getTradeOrders').content)['tradeOrders']
ordersList

[{'comissionAmount': 2.0,
  'get': 'zsh',
  'getVol': 10.0,
  'price': 25.25,
  'send': 'usdt',
  'sendVol': 252.5,
  'sender': 'z01x005ea5ca02a87f2dcf8685c8ae053d2cafb4c1e8b7f0519fb6676e1921',
  'symbol': 'zsh/usdt',
  'timestamp': 1630498714.252787,
  'tradeTxId': '9b998ddead8d4349bdb776b89e8274309602dea347c814ad8f6c5bfe10b40806'}]

In [15]:
login = requests.post(host+'/wallet/login', json={'password': '12345'}).content

In [23]:
sellTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 2.0,
        'send': 'zsh',
        'sendVol': 6.0,
        'get': 'usdt',
        'getVol': 12.0,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=sellTx).content


mineReq = requests.get(host+'/mine').content
mineReq

b'{"index":37,"message":"New Block Forged","pool_syncing":"Tx pool synced among 0 nodes","previous_hash":"375773c14fa6dc452f506b7387b4dbb6d22357b93efac7c8cfa70a882c65fd6d","proof":11,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"z01x008d58f8df5963a8bf0790264c612280987253dc0af0cb55897423f832","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1630498353.197109,"tradeTxId":null},{"comissionAmount":0.003333333333333333,"contract":"zsh","price":2.0,"recieveAmount":2.0,"recipient":"z01x00489ed1c516130d5021eb8d4582828bee0ec861b0f97febd73c66ecef","sendAmount":1.0,"sender":"z01x008d58f8df5963a8bf0790264c612280987253dc0af0cb55897423f832","symbol":"zsh/usdt","timestamp":1630498353.197181,"tradeTxId":"5183cdea4821883677ef4671e5351ec902cf5108c10b0afd0bf4323bfcb79686"},{"comissionAmount":0.004479283314669653,"contract":"usdt","price":2.0,"recieveAmount":1.0,"recipient":"z01x008d58

In [67]:
sybmolSortedLst = sorted([(ordersList[i]['sender'],ordersList[i]['symbol'],\
                            ordersList[i]['send'],ordersList[i]['get'],\
                           ordersList[i]['price'],ordersList[i]['tradeTxId'],)\
                          for i in range(len(ordersList))], key=itemgetter(4, 2, 1))
len(sybmolSortedLst)

20

In [65]:
import re
from operator import itemgetter

def matchOrders(ordersPool):


    sybmolSortedLst = sorted([(ordersPool[i]['sender'],ordersPool[i]['symbol'],\
                            ordersPool[i]['send'],ordersPool[i]['get'],\
                           ordersPool[i]['price'],ordersPool[i]['tradeTxId'],)\
                          for i in range(len(ordersPool))], key=itemgetter(4, 2, 1))

    mathchedOrders = []
    c = 0
    for i in range(int(len(sybmolSortedLst)/2)):
        if (sybmolSortedLst[c][2] == sybmolSortedLst[c+1][3]) and (sybmolSortedLst[c][3] == sybmolSortedLst[c+1][2]) :

            # Split symbol and define direction and price matching
            splittedSymbol1 = re.split(r'/', sybmolSortedLst[c][1])[0]
            splittedSymbol2 = re.split(r'/', sybmolSortedLst[c+1][1])[0]

            if ((splittedSymbol1 == sybmolSortedLst[c][2] and splittedSymbol2 == sybmolSortedLst[c+1][3]) \
                and (sybmolSortedLst[c][4] <= sybmolSortedLst[c+1][4])) \
                or ((splittedSymbol1 == sybmolSortedLst[c][3] and splittedSymbol2 == sybmolSortedLst[c+1][2])\
                   and (sybmolSortedLst[c][4] >= sybmolSortedLst[c+1][4])):

                mathchedOrders.append((sybmolSortedLst[c], sybmolSortedLst[c+1],))
                c+=2

    return mathchedOrders


In [69]:
mathchedOrders = matchOrders(ordersList)
mathchedOrders

[(('z01x004899db1c170b7eda76a739b24998936afe833edb17c0f0f83a7ee756',
   'zsh/usdt',
   'usdt',
   'zsh',
   9.93,
   '2472f804b268232fd836642a5e3aef6212ba00727aed730ef358b244f2d82fea'),
  ('z01x000cfb7e8773487fe02d9c9ecac125c29e95a4f2a64e19ffb17225f7e5',
   'zsh/usdt',
   'zsh',
   'usdt',
   9.93,
   '70e7ae90ae2559a1d93fb9d02bf835d3944a6bc346b948a198008bc3e7d0f916')),
 (('z01x004899db1c170b7eda76a739b24998936afe833edb17c0f0f83a7ee756',
   'zsh/usdt',
   'usdt',
   'zsh',
   10.93,
   'c510fa4bd53d5c6181d680bbc8a67be1a7430ff5f6643a5e2c4465774c8d618f'),
  ('z01x000cfb7e8773487fe02d9c9ecac125c29e95a4f2a64e19ffb17225f7e5',
   'zsh/usdt',
   'zsh',
   'usdt',
   10.93,
   '89fb43e19e27203df6e1fc159d5f4c28d818f96f2fa5e527146cb345ea031622')),
 (('z01x004899db1c170b7eda76a739b24998936afe833edb17c0f0f83a7ee756',
   'zsh/usdt',
   'usdt',
   'zsh',
   11.93,
   'db3f7b04943d911a39f4b78a1af5f127a78d50d3b2e276e8bce9ea6390a8d84a'),
  ('z01x000cfb7e8773487fe02d9c9ecac125c29e95a4f2a64e19ffb17225f7e

In [61]:
tempLst = [mathchedOrders[1][0][-1], mathchedOrders[1][1][-1]]
tempLst

['70e7ae90ae2559a1d93fb9d02bf835d3944a6bc346b948a198008bc3e7d0f916',
 'c510fa4bd53d5c6181d680bbc8a67be1a7430ff5f6643a5e2c4465774c8d618f']

In [59]:
txDir = {}
for i in range(len(mathchedOrders)):
    tempLst = [mathchedOrders[i][0][-1], mathchedOrders[i][1][-1]]
    print(tempLst)
    txTempLst=[order for order in ordersList if order['tradeTxId'] in tempLst]
    txDir[f'tx_{i}'] = txTempLst
txDir

['2472f804b268232fd836642a5e3aef6212ba00727aed730ef358b244f2d82fea', '70e7ae90ae2559a1d93fb9d02bf835d3944a6bc346b948a198008bc3e7d0f916']
['70e7ae90ae2559a1d93fb9d02bf835d3944a6bc346b948a198008bc3e7d0f916', 'c510fa4bd53d5c6181d680bbc8a67be1a7430ff5f6643a5e2c4465774c8d618f']
['c510fa4bd53d5c6181d680bbc8a67be1a7430ff5f6643a5e2c4465774c8d618f', '89fb43e19e27203df6e1fc159d5f4c28d818f96f2fa5e527146cb345ea031622']
['89fb43e19e27203df6e1fc159d5f4c28d818f96f2fa5e527146cb345ea031622', 'db3f7b04943d911a39f4b78a1af5f127a78d50d3b2e276e8bce9ea6390a8d84a']
['db3f7b04943d911a39f4b78a1af5f127a78d50d3b2e276e8bce9ea6390a8d84a', '93028616d4f7dad3020e83cbabf1b4c8baf975e1a78c50a5f8964955b20dca78']
['93028616d4f7dad3020e83cbabf1b4c8baf975e1a78c50a5f8964955b20dca78', '4fa3d0c963971386f2d96daba809fb4ca65fadf2271829f7b9da03e6c973b5b5']
['4fa3d0c963971386f2d96daba809fb4ca65fadf2271829f7b9da03e6c973b5b5', 'ca5bc3497dfa7b04ff6065109a342c5dbebb61d309d6b566f845052d20b2b785']
['ca5bc3497dfa7b04ff6065109a342c5dbebb61d

{'tx_0': [{'comissionAmount': 2.0,
   'get': 'usdt',
   'getVol': 9.93,
   'price': 9.93,
   'send': 'zsh',
   'sendVol': 1.0,
   'sender': 'z01x000cfb7e8773487fe02d9c9ecac125c29e95a4f2a64e19ffb17225f7e5',
   'symbol': 'zsh/usdt',
   'timestamp': 1630408730.301943,
   'tradeTxId': '70e7ae90ae2559a1d93fb9d02bf835d3944a6bc346b948a198008bc3e7d0f916'},
  {'comissionAmount': 2.0,
   'get': 'zsh',
   'getVol': 1.0,
   'price': 9.93,
   'send': 'usdt',
   'sendVol': 9.93,
   'sender': 'z01x004899db1c170b7eda76a739b24998936afe833edb17c0f0f83a7ee756',
   'symbol': 'zsh/usdt',
   'timestamp': 1630408730.942023,
   'tradeTxId': '2472f804b268232fd836642a5e3aef6212ba00727aed730ef358b244f2d82fea'}],
 'tx_1': [{'comissionAmount': 2.0,
   'get': 'usdt',
   'getVol': 9.93,
   'price': 9.93,
   'send': 'zsh',
   'sendVol': 1.0,
   'sender': 'z01x000cfb7e8773487fe02d9c9ecac125c29e95a4f2a64e19ffb17225f7e5',
   'symbol': 'zsh/usdt',
   'timestamp': 1630408730.301943,
   'tradeTxId': '70e7ae90ae2559a1d93fb9

In [49]:
jTxDir0 = json.dumps(txDir['tx_0'])
jTxDir1 = json.dumps(txDir['tx_1'])
jTxDir0

'[{"comissionAmount": 2.0, "get": "usdt", "getVol": 9.93, "price": 9.93, "send": "zsh", "sendVol": 1.0, "sender": "z01x000cfb7e8773487fe02d9c9ecac125c29e95a4f2a64e19ffb17225f7e5", "symbol": "zsh/usdt", "timestamp": 1630408730.301943, "tradeTxId": "70e7ae90ae2559a1d93fb9d02bf835d3944a6bc346b948a198008bc3e7d0f916"}, {"comissionAmount": 2.0, "get": "zsh", "getVol": 1.0, "price": 9.93, "send": "usdt", "sendVol": 9.93, "sender": "z01x004899db1c170b7eda76a739b24998936afe833edb17c0f0f83a7ee756", "symbol": "zsh/usdt", "timestamp": 1630408730.942023, "tradeTxId": "2472f804b268232fd836642a5e3aef6212ba00727aed730ef358b244f2d82fea"}]'

In [52]:
jTxDir1

'[{"comissionAmount": 2.0, "get": "usdt", "getVol": 9.93, "price": 9.93, "send": "zsh", "sendVol": 1.0, "sender": "z01x000cfb7e8773487fe02d9c9ecac125c29e95a4f2a64e19ffb17225f7e5", "symbol": "zsh/usdt", "timestamp": 1630408730.301943, "tradeTxId": "70e7ae90ae2559a1d93fb9d02bf835d3944a6bc346b948a198008bc3e7d0f916"}, {"comissionAmount": 2.0, "get": "zsh", "getVol": 1.0, "price": 10.93, "send": "usdt", "sendVol": 10.93, "sender": "z01x004899db1c170b7eda76a739b24998936afe833edb17c0f0f83a7ee756", "symbol": "zsh/usdt", "timestamp": 1630408730.986259, "tradeTxId": "c510fa4bd53d5c6181d680bbc8a67be1a7430ff5f6643a5e2c4465774c8d618f"}]'

In [58]:
if txDir['tx_2'][0]['tradeTxId'] != txDir['tx_4'][0]['tradeTxId']:
    print('yep')

yep


In [62]:
commonTxs = []
# Get matched orders
txDir = {}
for i in range(len(mathchedOrders)):
    tempLst = [mathchedOrders[i][0][-1], mathchedOrders[i][1][-1]]
    txTempLst=[order for order in ordersList if order['tradeTxId'] in tempLst]
    txDir[f'tx_{i}'] = txTempLst
    
txTempLst

[{'comissionAmount': 2.0,
  'get': 'usdt',
  'getVol': 18.93,
  'price': 18.93,
  'send': 'zsh',
  'sendVol': 1.0,
  'sender': 'z01x00798156877de01429deab82ed8fdf10c75d9fd97a3da7f8dd3dcbb15e',
  'symbol': 'zsh/usdt',
  'timestamp': 1630343483.237646,
  'tradeTxId': '4a9b37f8364e92088bd9fb98806b728127334b2fb72ed677002f3c2aef64eb61'},
 {'comissionAmount': 2.0,
  'get': 'zsh',
  'getVol': 1.0,
  'price': 18.93,
  'send': 'usdt',
  'sendVol': 18.93,
  'sender': 'z01x009056545cc1a516f2598268ef8f8067e153d476b61aa77273b5b725b2',
  'symbol': 'zsh/usdt',
  'timestamp': 1630343483.803811,
  'tradeTxId': '5275827ca03986ce1da80f79e86f93c56e5991aaab8df029aa078c7b0a12f4c7'}]

In [64]:
txDir

{'tx_0': [{'comissionAmount': 2.0,
   'get': 'usdt',
   'getVol': 9.93,
   'price': 9.93,
   'send': 'zsh',
   'sendVol': 1.0,
   'sender': 'z01x00798156877de01429deab82ed8fdf10c75d9fd97a3da7f8dd3dcbb15e',
   'symbol': 'zsh/usdt',
   'timestamp': 1630343482.831313,
   'tradeTxId': '2c13357a160db5e52b4f0b686e3ad47311a2c73407521017f6324c65df36e92b'},
  {'comissionAmount': 2.0,
   'get': 'zsh',
   'getVol': 1.0,
   'price': 9.93,
   'send': 'usdt',
   'sendVol': 9.93,
   'sender': 'z01x009056545cc1a516f2598268ef8f8067e153d476b61aa77273b5b725b2',
   'symbol': 'zsh/usdt',
   'timestamp': 1630343483.399211,
   'tradeTxId': '71d4ee13d94c461cf50dcf9f438181c98fa9ad0ca5b2f34c350b2b4e819534ee'}],
 'tx_1': [{'comissionAmount': 2.0,
   'get': 'usdt',
   'getVol': 9.93,
   'price': 9.93,
   'send': 'zsh',
   'sendVol': 1.0,
   'sender': 'z01x00798156877de01429deab82ed8fdf10c75d9fd97a3da7f8dd3dcbb15e',
   'symbol': 'zsh/usdt',
   'timestamp': 1630343482.831313,
   'tradeTxId': '2c13357a160db5e52b4f0b

In [63]:
txKeys = list(txDir.keys())
txKeys

['tx_0',
 'tx_1',
 'tx_2',
 'tx_3',
 'tx_4',
 'tx_5',
 'tx_6',
 'tx_7',
 'tx_8',
 'tx_9',
 'tx_10',
 'tx_11',
 'tx_12',
 'tx_13',
 'tx_14',
 'tx_15',
 'tx_16',
 'tx_17',
 'tx_18']

In [65]:
txDir[txKeys[0]][0]['sendVol'] -= sellSendAmount
txDir[txKeys[0]][0]['getVol'] -= buySendAmount
txDir[txKeys[0]][1]['sendVol'] -= buySendAmount
txDir[txKeys[0]][1]['getVol'] -= sellSendAmount

NameError: name 'sellSendAmount' is not defined

In [18]:
mineReq = requests.get(host+'/mine').content
mineReq

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'

In [37]:
mineReq = requests.get(host+'/mine').content
mineReq

b'{"index":22,"message":"New Block Forged","pool_syncing":"Tx pool synced among 0 nodes","previous_hash":"0bc6bc4871f7d9547d8c0793fa09436c5c64ce1477b4a856ec880ce74a73701a","proof":25,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"z01x00b3f5f21363fd6583cb158ba2a53b38b9786651d41a781cdb7a257e96","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1630165671.514097,"tradeTxId":null}]}\n'

### Minig speed testing

In [35]:
def miningSpeedTest():
    for i in range(10):
        requests.get('http://0.0.0.0:5000/mine')

In [36]:
cProfile.run('miningSpeedTest()', sort='time')

         52704 function calls (52694 primitive calls) in 19.547 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       28   19.483    0.696   19.483    0.696 {method 'recv_into' of '_socket.socket' objects}
      130    0.004    0.000    0.005    0.000 shlex.py:129(read_token)
       10    0.003    0.000    0.003    0.000 {built-in method _scproxy._get_proxy_settings}
     3000    0.003    0.000    0.008    0.000 os.py:673(__getitem__)
     3160    0.002    0.000    0.013    0.000 _collections_abc.py:742(__iter__)
       10    0.002    0.000    0.002    0.000 {built-in method _scproxy._get_proxies}
     5900    0.002    0.000    0.002    0.000 {method 'decode' of 'bytes' objects}
       10    0.002    0.000    0.002    0.000 {method 'connect' of '_socket.socket' objects}
       20    0.002    0.000    0.015    0.001 request.py:2456(getproxies_environment)
     3000    0.002    0.000    0.003    0.000 os.py:751(encode)
     

       10    0.000    0.000    0.000    0.000 models.py:61(path_url)
       20    0.000    0.000    0.000    0.000 {function SocketIO.close at 0x7f94ff9b0c80}
       50    0.000    0.000    0.000    0.000 message.py:479(set_raw)
       10    0.000    0.000    0.002    0.000 poolmanager.py:208(connection_from_host)
       50    0.000    0.000    0.000    0.000 _internal_utils.py:14(to_native_string)
       10    0.000    0.000    0.000    0.000 {method 'readinto' of '_io.BufferedReader' objects}
       10    0.000    0.000    0.000    0.000 parse.py:599(unquote)
       10    0.000    0.000    0.000    0.000 parse.py:762(quote)
       10    0.000    0.000    0.002    0.000 feedparser.py:173(feed)
       60    0.000    0.000    0.000    0.000 {method 'lstrip' of 'str' objects}
       20    0.000    0.000    0.000    0.000 socket.py:97(_intenum_converter)
       10    0.000    0.000    0.000    0.000 connectionpool.py:64(__init__)
       10    0.000    0.000    0.000    0.000 connectionpoo

### Test new pool creation  
Pool creation has to be a transaction with some zsh spenditure!

In [232]:
# Coinbase
wallet = requests.post(host+'/wallet/new', json={'password': '12345'}).content
wallet

b'{"ADDRESS":"z01x00677ed3eb9789e5405d94e59129389ffae05b41027ffc8af28942d89a"}\n'

In [233]:
wallet = requests.post(host+'/wallet/new', json={'password': 'qwerty'}).content
wallet

b'{"ADDRESS":"z01x00d6718ba1a1de9f995d9f6d7802089438a6e72cb9f5324a684c41be3c"}\n'

In [234]:
name = 'Tether' #Tether Ether
symbol = 'usdt' #usdt eth
pool = requests.post(host+'/pools/createPool', json={'name': name, 'symbol': symbol}).content
pool

b'{"MSG":"New pool was created! Name: Tether, Symbol: USDT"}\n'

In [38]:
pools = requests.get(host+'/pools/getPools').content
pools

b'{"MSG": [{"name": "Tether", "symbol": "USDT", "poolBalance": 0.0, "accounts": {"z01x00b3f5f21363fd6583cb158ba2a53b38b9786651d41a781cdb7a257e96": 10099.300000000003, "z01x00d546826798ee17af7f35f83ffe27d7b714414925471837326de9dd48": 9900.699999999997}, "blockHash": "136bfbb1d6476de0599b67d37b329af9590723a711ccd16565b0c60b1efed9c8", "validHash": "4d6e6149c371d4f73033aaa8832e00beb693d2b76a18a28d2377a8b9"}]}'

In [2]:
uid = uuid.uuid4().hex
password = 'lalalend'
passHashed = hashlib.sha3_224((password+uid).encode()).hexdigest()
passHashed

'2f1d2c396fe1aeec398cee910d55c44f254e470ee7a927e4890bcd7a'

In [28]:
whiteIp = requests.get('https://api.ipify.org').content
whiteIp.decode()

'82.151.196.144'

In [27]:
requests.get('http://178.176.120.241:5000').content

b'{"MSG":"Working"}\n'

## Test Blockchain Endpoints

In [2]:
host = 'http://0.0.0.0:5000'

In [3]:
nodes = requests.get(host+"/nodes/getNodes").content
nodes

b'{"nodes": ["178.176.120.241:5000"]}'

In [277]:
host1 = 'http://178.176.120.241:5000'
nodes1 = requests.get(host1+"/nodes/getNodes").content
nodes1

b'{"nodes": ["82.151.196.144:5000"]}'

In [115]:
wallet = json.loads(wallet)['ADDRESS']
wallet

'z01x006cd240bb549be12078aa8cf76dac965060dd2b6ba2f4151b6b0fe680'

In [12]:
regNode = requests.post(host+"/nodes/register", json={"nodes": ['http://0.0.0.0:5001']}).content
regNode

In [69]:
# logout = requests.get(host+'/wallet/logout').content
# logout

b'{"MSG":true}\n'

In [7]:
# Coinbase
wallet = requests.post(host+'/wallet/new', json={'password': '12345'}).content
wallet

b'{"ADDRESS":"z01x00bbf722aaa7c08bef5f20ca5e7afa62560a7ea3dcf68bd9d7745fbadc"}\n'

In [12]:
wallet = requests.post(host+'/wallet/new', json={'password': 'qwerty'}).content
wallet

b'{"ADDRESS":"z01x0011b6b1f87a66cf63d8e61d59429468dcb67d1324aebfd3f7bb51cc92"}\n'

In [8]:
login = requests.post(host+'/wallet/login', json={'password': '12345'}).content
login

b'{"ADDRESS":"z01x00bbf722aaa7c08bef5f20ca5e7afa62560a7ea3dcf68bd9d7745fbadc","MSG":true}\n'

In [9]:
mineReq = requests.get(host+'/mine').content
mineReq

b'{"index":2,"message":"New Block Forged","pool_syncing":"Tx pool synced among 1 nodes","previous_hash":"f64dedb7482a62da9fb55f718474f55a518f57cefa0e5b84014abc8a1290c316","proof":16,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"z01x00bbf722aaa7c08bef5f20ca5e7afa62560a7ea3dcf68bd9d7745fbadc","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1630303668.51556,"tradeTxId":null}]}\n'

In [238]:
accountBalance = requests.post(host+'/wallet/getBalance', json={'address': json.loads(login)['ADDRESS']}).content
accountBalance

b'{"BALACNE":{"balance":1.0,"token":"ZSH"}}\n'

In [239]:
sellTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 9.93,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 9.93,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=sellTx).content
txReq

b'{"MSG":3,"SYNC":"Tx pool synced among 0 nodes"}\n'

In [240]:
login = requests.post(host+'/wallet/login', json={'password': 'qwerty'}).content
login

b'{"ADDRESS":"z01x00d6718ba1a1de9f995d9f6d7802089438a6e72cb9f5324a684c41be3c","MSG":true}\n'

In [241]:
accountBalance = requests.post(host+'/wallet/getBalance', json={'address': json.loads(login)['ADDRESS']}).content
accountBalance

b'{"BALACNE":{"balance":0.0,"token":"ZSH"}}\n'

In [242]:
buyTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 9.93,
        'send': 'usdt',
        'sendVol': 9.93,
        'get': 'zsh',
        'getVol': 1.0,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=buyTx).content

txReq

b'{"MSG":3,"SYNC":"Tx pool synced among 0 nodes"}\n'

In [243]:
mineReq = requests.get(host+'/mine').content
mineReq

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'

In [190]:
accountBalance = requests.post(host+'/wallet/getBalance', json={'address': json.loads(login)['ADDRESS']}).content
accountBalance

b'{"BALACNE":{"balance":2.0,"token":"ZSH"}}\n'

In [130]:
step=0.1
for i in range(3):
    
    tradeTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 10.0+step,
        'send': 'zsh',
        'sendVol': 1.0,
        'get': 'usdt',
        'getVol': 10.0+step,
        'comissionAmount':2
    }

    txReq = requests.post(host+'/transactions/new', json=tradeTx).content
    step += 0.1

txReq

b'{"MSG":4,"SYNC":"Tx pool synced among 1 nodes"}\n'

In [131]:
step = 0.1
for i in range(3):
    tradeTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'zsh/usdt',
        'price': 9.8+step,
        'send': 'usdt',
        'sendVol': 9.8+step,
        'get': 'zsh',
        'getVol': 1.0,
        'comissionAmount':2
    }
    
    txReq = requests.post(host+'/transactions/new', json=tradeTx).content
    step -= 0.1
txReq

b'{"MSG":4,"SYNC":"Tx pool synced among 1 nodes"}\n'

In [36]:
ordersPool = json.loads(requests.get(host+'/getTradeOrders').content)['tradeOrders']
ordersPool

[{'comissionAmount': 2.0,
  'get': 'usdt',
  'getVol': 10.1,
  'price': 10.1,
  'send': 'zsh',
  'sendVol': 1.0,
  'sender': 'z01x006cd240bb549be12078aa8cf76dac965060dd2b6ba2f4151b6b0fe680',
  'symbol': 'zsh/usdt',
  'timestamp': 1629980429.780089,
  'tradeTxId': 'cc9828d5acda1758fc2435ad279c436213bd5d65a6767e557d9b0f9c88794a9c'},
 {'comissionAmount': 2.0,
  'get': 'usdt',
  'getVol': 10.2,
  'price': 10.2,
  'send': 'zsh',
  'sendVol': 1.0,
  'sender': 'z01x006cd240bb549be12078aa8cf76dac965060dd2b6ba2f4151b6b0fe680',
  'symbol': 'zsh/usdt',
  'timestamp': 1629980430.591604,
  'tradeTxId': 'f82a51ea6f4e22fd10c4984a9b20965774e4f308eeae3d039dea659034c03d41'},
 {'comissionAmount': 2.0,
  'get': 'usdt',
  'getVol': 10.3,
  'price': 10.3,
  'send': 'zsh',
  'sendVol': 1.0,
  'sender': 'z01x006cd240bb549be12078aa8cf76dac965060dd2b6ba2f4151b6b0fe680',
  'symbol': 'zsh/usdt',
  'timestamp': 1629980431.33705,
  'tradeTxId': '040d642e897cf24f6784de7ccdc1700bcf8f56881aa1d80751f65c2da2105573'},
 {

In [132]:
chain = json.loads(requests.get(host+'/chain').content)['chain']
chain

[{'index': 1,
  'previousHash': '9c3b78de7eb0066418a682aec7d0e6841cd5cbe70d71ae65309d4917cd1c5681',
  'proof': 42,
  'timestamp': 1629032178.368946,
  'transactions': ['ascl,sdc,d[v,[d,vsdcaaxasx]]', 'lv,sldc,sdc ;,d;d']},
 {'index': 2,
  'previousHash': '9204ca3bc9ca2716f5a0a3ed866c9fb5b15a292c0ea53bc2e3e145b94933f393',
  'proof': 22976,
  'timestamp': 1630062487.100557,
  'transactions': [{'comissionAmount': 1.0,
    'contract': None,
    'price': 0.0,
    'recieveAmount': None,
    'recipient': 'z01x0028f3a682c732e293aad870ea7ed6bdd79b0fca340cf6927cf9e9c10a',
    'sendAmount': 1,
    'sender': 'z01x0000000000000000000000000000000000000000000000000000000000',
    'symbol': 'zsh',
    'timestamp': 1630062486.249603,
    'tradeTxId': None}]},
 {'index': 3,
  'previousHash': '422699de7eaf9da0437a4e67a69bbc18192eef6d4d145112fac4caaa8c72826b',
  'proof': 119240,
  'timestamp': 1630062491.103936,
  'transactions': [{'comissionAmount': 1.0,
    'contract': None,
    'price': 0.0,
    'recie

In [71]:
# 'comissionAmount': 2.0,
#   'get': 'usdt',
#   'getVol': 1030.0,
#   'price': 1030.0,
#   'send': 'uni',
#   'sendVol': 1.0,
#   'sender': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
#   'symbol': 'uni/usdt',
#   'timestamp': 1629813262.264664,
#   'tradeTxId': 'd648507f2b334c965c3e616f6d06fb1e987e820453385b8e65183c2f994265c3'}


buyTx = {
        'type': 'trade',
        'sender': json.loads(login)['ADDRESS'],
        'symbol': 'uni/usdt',
        'price': 1000.0,
        'send': 'usdt',
        'sendVol': 1000.0,
        'get': 'uni',
        'getVol': 1.0,
        'comissionAmount':2
    }
txReq = requests.post(host+'/transactions/new', json=buyTx).content


# {'comissionAmount': 2.0,
#   'get': 'usdt',
#   'getVol': 49010.0,
#   'price': 49010.0,
#   'send': 'btc',
#   'sendVol': 1.0,
#   'sender': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
#   'symbol': 'btc/usdt',
#   'timestamp': 1629813003.165863,
#   'tradeTxId': 'b0c04cc31ff140a01b7bacaad8dd8b866b93861edc74e4e2be28e0327e31c139'}

# buyTx = {
#         'type': 'trade',
#         'sender': json.loads(login)['ADDRESS'],
#         'symbol': 'btc/usdt',
#         'price': 49010.1,
#         'send': 'usdt',
#         'sendVol': 49010.1,
#         'get': 'btc',
#         'getVol': 1.0,
#         'comissionAmount':2
#     }
txReq = requests.post(host+'/transactions/new', json=buyTx).content
txReq

b'{"MSG":4,"SYNC":"Tx pool synced among 1 nodes"}\n'

In [48]:
ordersPool = json.loads(requests.get(host+'/getTradeOrders').content)['tradeOrders']
ordersPool

[{'comissionAmount': 2.0,
  'get': 'usdt',
  'getVol': 1010.0,
  'price': 1010.0,
  'send': 'uni',
  'sendVol': 1.0,
  'sender': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'symbol': 'uni/usdt',
  'timestamp': 1629891726.53693,
  'tradeTxId': '562bea1978aa9504102791d5b466ea68e40cf62605659248170a5ed150e53857'},
 {'comissionAmount': 2.0,
  'get': 'usdt',
  'getVol': 1020.0,
  'price': 1020.0,
  'send': 'uni',
  'sendVol': 1.0,
  'sender': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'symbol': 'uni/usdt',
  'timestamp': 1629891727.30612,
  'tradeTxId': 'cdbb497b923da5f1e8c473e2e4d74e0f78ff0b7897c728798c4c0b7ae36218c0'},
 {'comissionAmount': 2.0,
  'get': 'usdt',
  'getVol': 1030.0,
  'price': 1030.0,
  'send': 'uni',
  'sendVol': 1.0,
  'sender': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'symbol': 'uni/usdt',
  'timestamp': 1629891728.041295,
  'tradeTxId': '5209e1a8ebe118fe1f25b27fcca9a03bca2012d81f5f40046ac99211138

In [26]:
chain = json.loads(requests.get(host+'/chain').content)['chain']
chain

[{'index': 1,
  'previousHash': '9c3b78de7eb0066418a682aec7d0e6841cd5cbe70d71ae65309d4917cd1c5681',
  'proof': 42,
  'timestamp': 1629032178.368946,
  'transactions': ['ascl,sdc,d[v,[d,vsdcaaxasx]]', 'lv,sldc,sdc ;,d;d']}]

In [38]:
sybmolSortedLst = sorted([(ordersPool[i]['sender'],ordersPool[i]['symbol'],\
                            ordersPool[i]['send'],ordersPool[i]['get'],\
                           ordersPool[i]['price'],ordersPool[i]['tradeTxId'],)\
                          for i in range(len(ordersPool))], key=itemgetter(4, 2, 1))

sybmolSortedLst

[('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'btc/usdt',
  'usdt',
  'btc',
  48960.0,
  '86e46974fdd63dd773da36f3724e91ec782e62110b44637fde6322f39d4101ce'),
 ('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'btc/usdt',
  'usdt',
  'btc',
  48970.0,
  'fd782f8c14c710d1e33ca276961e9cb7a61fff371ead383dac2d81ddb15af1a0'),
 ('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'btc/usdt',
  'usdt',
  'btc',
  48980.0,
  '6aab946489976021faff63a985d37243a65b1ec0c3c837bc0d92c6b1154aae3f'),
 ('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'btc/usdt',
  'btc',
  'usdt',
  49000.0,
  '6d348eea99de4b9a0999f1b252cdf1c28d8bba0bdecf3184c17f64291e2ab4e3'),
 ('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'btc/usdt',
  'usdt',
  'btc',
  49000.0,
  '6356596306c008b19dee84b58701b0d93bdd07b17160c6caf24ce4314082b511'),
 ('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
  'btc/usdt',
  '

In [39]:
mathchedOrders=[]
for i in range(len(sybmolSortedLst)-1):
    if (sybmolSortedLst[i][2] == sybmolSortedLst[i+1][3]) and (sybmolSortedLst[i][3] == sybmolSortedLst[i+1][2]) :

        # Split symbol and define direction and price matching
        splittedSymbol1 = re.split(r'/', sybmolSortedLst[i][1])[0]
        splittedSymbol2 = re.split(r'/', sybmolSortedLst[i+1][1])[0]

        if ((splittedSymbol1 == sybmolSortedLst[i][2] and splittedSymbol2 == sybmolSortedLst[i+1][3]) \
            and (sybmolSortedLst[i][4] <= sybmolSortedLst[i+1][4])) \
            or ((splittedSymbol1 == sybmolSortedLst[i][3] and splittedSymbol2 == sybmolSortedLst[i+1][2])\
               and (sybmolSortedLst[i][4] >= sybmolSortedLst[i+1][4])):

            mathchedOrders.append((sybmolSortedLst[i], sybmolSortedLst[i+1],))
mathchedOrders

[(('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
   'btc/usdt',
   'btc',
   'usdt',
   49000.0,
   '6d348eea99de4b9a0999f1b252cdf1c28d8bba0bdecf3184c17f64291e2ab4e3'),
  ('z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
   'btc/usdt',
   'usdt',
   'btc',
   49000.0,
   '6356596306c008b19dee84b58701b0d93bdd07b17160c6caf24ce4314082b511'))]

In [56]:
mathchedOrders[0][1][-1]

'6356596306c008b19dee84b58701b0d93bdd07b17160c6caf24ce4314082b511'

In [ ]:
commonTxs = []
# Get matched orders
txDir = {}
for i in range(len(mathchedOrders)):
    tempLst = [mathchedOrders[i][0][0], mathchedOrders[i][1][0]]
    txTempLst=[order for order in ordersList if order['tradeTxId'] in tempLst]
    txDir[f'tx_{i}'] = txTempLst

In [270]:
txPool = requests.get(host+'/getTxPool').content
txPool

b'{"txPool":[]}\n'

In [67]:
chain = json.loads(requests.get(host+'/chain').content)['chain']
chain

[{'index': 1,
  'previousHash': '9c3b78de7eb0066418a682aec7d0e6841cd5cbe70d71ae65309d4917cd1c5681',
  'proof': 42,
  'timestamp': 1629032178.368946,
  'transactions': ['ascl,sdc,d[v,[d,vsdcaaxasx]]', 'lv,sldc,sdc ;,d;d']},
 {'index': 2,
  'previousHash': '9204ca3bc9ca2716f5a0a3ed866c9fb5b15a292c0ea53bc2e3e145b94933f393',
  'proof': 22976,
  'timestamp': 1629891791.715504,
  'transactions': [{'comissionAmount': 1.0,
    'contract': None,
    'price': 0.0,
    'recieveAmount': None,
    'recipient': '97c3a1fcdbad4085b059b22735c74faf',
    'sendAmount': 1,
    'sender': 'z01x0000000000000000000000000000000000000000000000000000000000',
    'symbol': 'zsh',
    'timestamp': 1629891791.043628,
    'tradeTxId': None},
   {'comissionAmount': 0.02,
    'contract': 'uni',
    'price': 1001.0,
    'recieveAmount': 1001.0,
    'recipient': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e',
    'sendAmount': 1.0,
    'sender': 'z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765

In [ ]:
chain[0][-1][]

### NODE 1

In [71]:
# host1 = 'http://0.0.0.0:5001'
host1 = 'http://178.176.120.241:5000'

In [74]:
wallet1 = requests.post(host1+'/wallet/new', json={'password': 'lalalend'}).content
wallet1

b'{"ADDRESS":"z01x001ddb88cc07c6d9179c51b370128d07109a650cba29c5889c078e6bb0"}\n'

In [266]:
login1 = requests.post(host1+'/wallet/login', json={'password': 'lalalend'}).content
login1

b'{"ADDRESS":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","MSG":true}\n'

In [267]:
tradeTx1 = {
    'type': 'trade',
    'sender': json.loads(login1)['ADDRESS'],
    'symbol': 'btc/usdt',
    'price': 3.0,
    'send': 'btc',
    'sendVol': 2.0,
    'get': 'usdt',
    'getVol': 6.0,
    'comissionAmount':2
}

txReq1 = requests.post(host1+'/transactions/new', json=tradeTx1).content
txReq1

b'{"MSG":[3,"Tx pool synced among 1 nodes"]}\n'

In [268]:
tradeTx1 = {
    'type': 'trade',
    'sender': json.loads(login1)['ADDRESS'],
    'symbol': 'btc/usdt',
    'price': 3.0,
    'send': 'usdt',
    'sendVol': 6.0,
    'get': 'btc',
    'getVol': 2.0,
    'comissionAmount':2
}

txReq1 = requests.post(host1+'/transactions/new', json=tradeTx1).content
txReq1

b'{"MSG":[3,"Tx pool synced among 1 nodes"]}\n'

In [72]:
ordersPool1 = requests.get(host1+'/getTradeOrders').content
ordersPool1

b'{"tradeOrders":[{"comissionAmount":2.0,"get":"usdt","getVol":49010.0,"price":49010.0,"send":"btc","sendVol":1.0,"sender":"z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e","symbol":"btc/usdt","timestamp":1629785450.250563,"tradeTxId":"097e6811fd1bb472cf2acc15bbcf773e224b10cf4c5a814201075b48eb455a1e"},{"comissionAmount":2.0,"get":"usdt","getVol":49020.0,"price":49020.0,"send":"btc","sendVol":1.0,"sender":"z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e","symbol":"btc/usdt","timestamp":1629785451.023485,"tradeTxId":"c31a89faee68430c5dbc117c60d35a45080e842b591b76a80e1fe1d32376f628"},{"comissionAmount":2.0,"get":"usdt","getVol":49030.0,"price":49030.0,"send":"btc","sendVol":1.0,"sender":"z01x0030dfc2c4736a6ecaa2362cb65f7fb313934b33283f51caa765e3745e","symbol":"btc/usdt","timestamp":1629785451.748514,"tradeTxId":"ba171caf3e650036595ee8432cb0627ceb1c2c0267fb03f9ac4a0218ac2679eb"},{"comissionAmount":2.0,"get":"btc","getVol":1.0,"price":48980.0,"send":"usdt","sen

In [272]:
mineReq1 = requests.get(host1+'/mine').content
mineReq1

b'{"index":3,"message":"New Block Forged","pool_syncing":"Tx pool synced among 1 nodes","previous_hash":"49c4fa4aada5dec35ce6f03f9de1465628b9513738ce8d3dd1f18cb17d0f2a8f","proof":119240,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"895cc4a0852d41a9a9c4f7f0ad55d296","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1629723377.434202,"tradeTxId":null},{"comissionAmount":2.0,"contract":"btc","price":3.0,"recieveAmount":6.0,"recipient":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","sendAmount":2.0,"sender":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","symbol":"btc/usdt","timestamp":1629723377.440518,"tradeTxId":"fb7456550faa82f198c6026e488ce59602ae9a013ad08fa808e2b6bdff76e856"},{"comissionAmount":2.0,"contract":"usdt","price":3.0,"recieveAmount":2.0,"recipient":"z01x0011c451ec98668981d667349895f26de90940f0b55a2843eabab0ebc7","sendAm

In [177]:
txPool1 = requests.get(host1+'/getTxPool').content
txPool1

b'{"txPool":[]}\n'

In [273]:
chain1 = requests.get(host1+'/chain').content
chain1

b'{"chain":[{"index":1,"previousHash":"9c3b78de7eb0066418a682aec7d0e6841cd5cbe70d71ae65309d4917cd1c5681","proof":42,"timestamp":1629032178.368946,"transactions":["ascl,sdc,d[v,[d,vsdcaaxasx]]","lv,sldc,sdc ;,d;d"]},{"index":2,"previousHash":"9204ca3bc9ca2716f5a0a3ed866c9fb5b15a292c0ea53bc2e3e145b94933f393","proof":22976,"timestamp":1629723151.057186,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"d32902ea13e44d3a9f905d99620d0892","sendAmount":1,"sender":"z01x0000000000000000000000000000000000000000000000000000000000","symbol":"zsh","timestamp":1629723151.043912,"tradeTxId":null},{"comissionAmount":2.0,"contract":"btc","price":30000.0,"recieveAmount":60000.0,"recipient":"z01x00daba2c45b077691c1ada65fb07f5e740e455d4463c59028b37055900","sendAmount":2.0,"sender":"z01x00daba2c45b077691c1ada65fb07f5e740e455d4463c59028b37055900","symbol":"btc/usdt","timestamp":1629723151.057158,"tradeTxId":"8e85b87e9298b3d39259884f5edc56eb516ec865d96333783c